# The University of Melbourne, School of Computing and Information Systems

-----
###### Student Name(s):yajing zhang(1100453)
###### Python version: 3

In [ ]:
import pandas as pd 
import numpy as np 
import math
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
# Implementation 

# load data
student =load_data("student.csv")
student.head()

# split data into train and test
train_data, test_data = split(student,0.8) #train data set percentage eg 0.8

# Data training
proList, gradeList = train(train_data,"Grade")

# predict data
predication= predict(proList, test_data, 'Grade', gradeList)

# model evaluation 
evaluate(predication['Grade'],predication['predict'])

In [10]:
# This function should open a data file in csv, and transform it into a usable format 
def load_data(fileName):
    
    return pd.read_csv(open(fileName))


In [5]:
# This function should split a data set into a training set and hold-out test set
def split(data, percentage):
#   calculate the size of testData and training Data
    length = len(data)
    trainDataLength= int(percentage*length)
    testDataLength=length-trainDataLength
    
#   generate testDataSet and trainingDataSet
    data = shuffle(data)
    data_train =data[0:trainDataLength]
    data_test = data[trainDataLength:length]
    data_train = data_train.reset_index()
    data_test = data_test.reset_index()
    
    return ( data_train, data_test)

In [6]:
# This function should build a supervised NB model
def train ( df,responseVariable):
    
    return probList(df,responseVariable) , gradeProb (df)
# Helper function return a list contain probability of each grade
def gradeProb (data):
    responseVariable ='Grade'
    
    gradeSet = list(set(data[responseVariable]))
    featureList = data.columns.values.tolist().remove(responseVariable)
    gradeProb = {}
    for grade in gradeSet:
        
        
        size = len(data)
        gradeNumber= len(student.loc[student[responseVariable]==grade])
        probability= gradeNumber/size
        gradeProb[grade]=probability

        
    return  gradeProb

# Helper function return a list of probability for each category of each feature for each grades
def probList (df,responseVariable):
    grades = list(set(df[responseVariable])) #number of possible grade outcome eg A+ A..
    result = {}
    
    features = df.columns.values
    features = features.tolist()
    features.remove( responseVariable) #list of feature exclude responseVariable
    features.remove('index')
    
    for grade in grades:
        f = {}
        result[grade] = []
        a = df.loc[(df[ responseVariable]==grade)] #list of instance for a specific grade
        gradeSize = len(a)# length of instance for a particular grade
       
       
        for feature in features:
            f[feature] = []
            data = pd.DataFrame(a[feature].value_counts())
            data = data.reset_index()
            ls1 = list(set(a[feature]))# list of possible feature result of a grade
            lsd = {}
            for j in ls1:

                for row in range(0,len(data)):
                    l = []
                    if  j == data.loc[row]["index"]:
                        t = data.loc[row][feature]
                        p =(t / gradeSize)
                        r = round(p,5)
                        l.append(r)
                        lsd.update({j:r})
                    else:
                        pass
                    
            f.update({feature:lsd})     
        result[grade] = f    
    return (result)

In [7]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(probList, testdata, col, gradeProb): 
    
    features = testdata.columns.values
    features = features.tolist()
    features.remove(col)
    features.remove('index')
    
    pred = []
    for row in range(0,len(testdata)): # loop through all instances
        ter = testdata.loc[row] 
        a = []
        aplus = []
        b=[]
        c=[]
        d=[]
        f=[]
        
        for feature in features: #loop through all features of each instance 
            cell = ter[feature] # the value of (row, feature)
            print(cell)
            print(row)
            print(feature)
            
            
            try:
                aplusList = probList['A+'][feature][cell] 
                
            except TypeError:
                aplusList = probList['A+'][feature][int(cell)] #else condition for integer cell
            except: 
                aplusList =0 
                aplus.append(aplusList)
        
        
            try:
                aList = probList['A'][feature][cell]
            except TypeError:
                aList = probList['A'][feature][int(cell)] #else condition for integer cell
            except:
                aList =0
            a.append(aList)
            
        
            
            try:
                bList = probList['B'][feature][cell]
            except TypeError:
                bList = probList['B'][feature][int(cell)] #else condition for integer cell
            except: 
                bList =0
            b.append(bList)
    
            
            try:
                cList = probList['C'][feature][cell]
            except TypeError:
                cList = probList['C'][feature][int(cell)] #else condition for integer cell
            except:
                cList =0   
            c.append(cList)
            try:
                dList = probList['D'][feature][cell]
            except TypeError:
                dList = probList['D'][feature][int(cell)] #else condition for integer cell
            except:
                dList =0   
            d.append(dList)
    
            
            try:
                fList = probList['F'][feature][cell]
            except TypeError:
                fList = probList['F'][feature][int(cell)] #else condition for integer cell
            except: 
                fList =0    
            f.append(fList)
            
            
        computeAPlus = (np.prod(aplusList))*gradeProb['A+']
        computeA = (np.prod(aList))*gradeProb['A']
        computeB = (np.prod(bList))*gradeProb['B']
        computeC = (np.prod(cList))*gradeProb['C']
        computeD = (np.prod(dList))*gradeProb['D']
        computeF = (np.prod(fList))*gradeProb['F']
        
        scoreList = [computeAPlus,computeA,computeB,computeC,computeD,computeF]
        maxElement= scoreList.index(max(scoreList))
        
        
        if maxElement ==0 : pred.append('A+') 
        elif maxElement ==1 : pred.append('A') 
        elif maxElement ==2 : pred.append('B')
        elif maxElement ==3 : pred.append('C') 
        elif maxElement ==4 : pred.append('D') 
        elif maxElement ==5 : pred.append('F') 
        else :pred.append('error') 
            
    testdata["predict"] = pred
    return testdata

In [8]:
# This function should evaluate a set of predictions in terms of accuracy
def evaluate(actual, predicted):
    
#     return accuracy , errror rate ,classificationReport
    return metrics.accuracy_score(actual, predicted), (1-metrics.accuracy_score(actual, predicted)),classification_report(actual, predicted)
